Grop Member:
Lei Li, Yichen Han, and Zihan Chen

# Data pre-procession

In [2]:
import os
import json


from google.colab import drive
drive.mount('/content/drive')



name_dict = {
    "apple": 0, "banana": 1, "guava": 2, "lime": 3, "orange": 4, "pomegranate": 5
}

# summary the the pictures in each category
name2data = {}

data_root_path = '/content/drive/My Drive/FruitDataSet/'
test_file_path = data_root_path + "test.list"
train_file_path = data_root_path + "train.list"
readme_file = data_root_path + "readme.json"


def save_train_test_file(path, name):
  if name not in name2data:
    name2data[name] = []

  name2data[name].append(path)


for d in os.listdir(data_root_path):
  # print(d)
  full_path = data_root_path + d
  if os.path.isdir(full_path):
    imgs = os.listdir(full_path)
    for img in imgs:
      save_train_test_file(full_path + "/" + img, d.lower())
  else:
    pass


# clear the file first
with open(test_file_path, 'w') as f:
  pass

with open(train_file_path, 'w') as f:
  pass

# now let's divide the data into train set and test set. 1 : 10
for name, imgs in name2data.items():
  print(name, "  ", len(imgs))
  n = len(imgs)
  n = 100
  for i in range(n):
    if i % 10:
      with open(train_file_path, 'a') as f:
        line = f"{imgs[i]}\t{name_dict[name]}\n"
        f.write(line)
    else:
      with open(test_file_path, 'a') as f:
        line = f"{imgs[i]}\t{name_dict[name]}\n"
        f.write(line)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
apple    1149
orange    1216
banana    1113
lime    1094
pomegranate    5940
guava    1152


# Model build and training

In [4]:
!pip install paddlepaddle-gpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.9/758.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00


In [4]:
pip install Pillow

In [5]:
import os
import numpy as np
from PIL import Image
import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import Dataset, DataLoader
import paddle.vision.transforms as T

data_root_path = '/content/drive/My Drive/FruitDataSet/'
test_file_path = data_root_path + "test.list"
train_file_path = data_root_path + "train.list"

# Transformation for image
transform = T.Compose([
    T.Resize((100, 100)), # Resizes the image to 100 x 100
    T.ToTensor(), # Converts to Tensor, scales to [0, 1] by dividing by 255
])

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, data_file):
        super(CustomDataset, self).__init__()
        self.data = []
        with open(data_file, 'r') as f:
            for line in f:
                img_path, label = line.strip().split('\t')
                self.data.append((img_path, int(label)))

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB') # Ensure image is RGB
        img = transform(img) # Apply transformations
        return img, label

    def __len__(self):
        return len(self.data)

# Define the model


class ConvolutionalNeuralNetwork(nn.Layer):
    def __init__(self, type_size):
        super(ConvolutionalNeuralNetwork, self).__init__()

        # Define the first convolutional pooling layer
        self.conv_pool_1 = nn.Sequential(
            nn.Conv2D(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=2, stride=2),
            nn.Dropout(p=0.5)
        )

        # Define the second convolutional pooling layer
        self.conv_pool_2 = nn.Sequential(
            nn.Conv2D(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=2, stride=2),
            nn.Dropout(p=0.5)
        )

        # Define the third convolutional pooling layer
        self.conv_pool_3 = nn.Sequential(
            nn.Conv2D(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2D(kernel_size=2, stride=2),
            nn.Dropout(p=0.5)
        )

        # Define the first fully connected layer
        self.fc1 = nn.Sequential(
            nn.Linear(in_features=9216, out_features=512),  # !!!`in_features`
            nn.ReLU(),
            nn.Dropout(p=0.5)
        )

        # Define the prediction fully connected layer
        self.fc2 = nn.Linear(in_features=512, out_features=type_size)

    def forward(self, image):
        x = self.conv_pool_1(image)
        x = self.conv_pool_2(x)
        x = self.conv_pool_3(x)

        # Flatten the output for the fully connected layer
        x = paddle.flatten(x, start_axis=1)

        # Apply the first fully connected layer and dropout
        x = self.fc1(x)

        # Apply the second fully connected layer for prediction
        x = self.fc2(x)

        # Apply softmax for prediction
        # x = nn.functional.softmax(x, axis=1)

        return x

# Initialize the model and optimizer
model = ConvolutionalNeuralNetwork(6)
optimizer = paddle.optimizer.Adam(learning_rate=0.001, parameters=model.parameters())

# Use GPU for training
paddle.set_device('gpu:0')

# Create the dataset and data loader
train_dataset = CustomDataset(train_file_path) # Make sure to provide the correct path
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

# Training loop
for epoch in range(10):
    for batch_id, (images, labels) in enumerate(train_loader):
        # Forward pass
        predicts = model(images)

        # Convert labels to tensor
        labels = paddle.to_tensor(labels, dtype='int64')

        # Calculate the loss
        loss = F.cross_entropy(predicts, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()

        if batch_id % 20 == 0:
            print(f"Epoch {epoch}, Batch {batch_id}, Loss {loss.numpy()}")

print('Training complete')


Epoch 0, Batch 0, Loss 7.4582366943359375
Epoch 1, Batch 0, Loss 1.8055946826934814
Epoch 2, Batch 0, Loss 1.7835028171539307
Epoch 3, Batch 0, Loss 1.768263816833496
Epoch 4, Batch 0, Loss 1.466644525527954
Epoch 5, Batch 0, Loss 0.70966637134552
Epoch 6, Batch 0, Loss 0.5495848655700684
Epoch 7, Batch 0, Loss 0.8474939465522766
Epoch 8, Batch 0, Loss 0.7155379056930542
Epoch 9, Batch 0, Loss 0.21582749485969543
Training complete


In [6]:
def validate(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_samples = 0
    loss_list = []

    for batch_id, (images, labels) in enumerate(data_loader):
        predicts = model(images)
        labels = paddle.to_tensor(labels, dtype='int64').unsqueeze(-1)  # Add a dimension to labels
        loss = F.cross_entropy(predicts, labels)
        loss_list.append(loss.numpy())

        # Calculate accuracy using Paddle's built-in function
        predicted_labels = paddle.argmax(predicts, axis=1).unsqueeze(-1)
        correct = paddle.equal(predicted_labels, labels).astype(paddle.int32)
        total_correct += paddle.sum(correct).numpy()
        total_samples += labels.shape[0]

        # Print image details
        predicted_labels = predicted_labels.numpy().flatten()
        actual_labels = labels.squeeze(-1).numpy()
        probabilities = predicts.numpy()
        for img_idx in range(len(images)):
            print(f"Image: {data_loader.dataset.data[batch_id * BATCH_SIZE + img_idx][0]}, "
                  f"Actual: {actual_labels[img_idx]}, "
                  f"Predicted: {predicted_labels[img_idx]}, "
                  f"Probabilities: {probabilities[img_idx]}")

    avg_loss = np.mean(loss_list)
    accuracy = total_correct / total_samples
    print(f"Total Correct: {total_correct}, Total Samples: {total_samples}, Calculated Accuracy: {accuracy}")
    return avg_loss, accuracy


BATCH_SIZE = 32
test_dataset = CustomDataset(test_file_path)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)


# After the training loop
avg_loss, accuracy = validate(model, test_loader)
print(f"Validation Loss: {avg_loss}, Accuracy: {accuracy}")


Image: /content/drive/My Drive/FruitDataSet/Apple/20190809_122017.jpg, Actual: 0, Predicted: 0, Probabilities: [ 3.2886198  -0.1497783   0.77602285  0.18880889  0.8292114  -0.88134474]
Image: /content/drive/My Drive/FruitDataSet/Apple/20190809_122140.jpg, Actual: 0, Predicted: 0, Probabilities: [ 7.8425803  1.2932645  6.874145  -5.5826545 -5.245053   1.621631 ]
Image: /content/drive/My Drive/FruitDataSet/Apple/20190809_122237.jpg, Actual: 0, Predicted: 0, Probabilities: [ 5.7204633   0.7394698   5.4821277  -4.7466736  -0.20485233  0.4223248 ]
Image: /content/drive/My Drive/FruitDataSet/Apple/20190809_122435.jpg, Actual: 0, Predicted: 0, Probabilities: [ 6.0690064   0.36053398  5.035973   -1.913502    1.714533   -2.8261306 ]
Image: /content/drive/My Drive/FruitDataSet/Apple/20190809_122710.jpg, Actual: 0, Predicted: 0, Probabilities: [ 5.3617005  -0.23567587  3.627666   -1.070859   -0.8148127  -1.7736454 ]
Image: /content/drive/My Drive/FruitDataSet/Apple/20190809_122752.jpg, Actual: 0,